In [1]:
from deltalake import DeltaTable
import numpy as np
import pandas as pd

bets = DeltaTable("/Users/andreikirpichev/parquet/bets/")
bets = bets.to_pandas()
bets.sort_values('placement_time', ascending=False)

,customer_id,age,country,section,point,market,market_params,contest_id,placement_date,placement_time,...,region,sport,league,minutes_before_start,stake_eur,dayofweek,weekday_stake,section_2w,point_2w,market_2w
3935682,3044310,40,AT,HT1,Goal,OverUnder,None,FB-226375,2022-10-28,2022-10-28 14:23:42.051,...,JO,Football,Premier League,0.0,10.86,6,22.20,21.46,20.31,20.74
3935681,17707010,60,DE,RT,Goal,OverUnder,None,FB-226334,2022-10-28,2022-10-28 14:23:40.631,...,EG,Football,2nd Division,0.0,20.00,6,33.00,12.84,13.01,13.09
3935680,11764610,61,DE,RT,Goal,Handicap3w,None,FB-226449,2022-10-28,2022-10-28 14:23:39.479,...,BG,Football,First League,66.0,0.50,6,0.25,0.25,0.25,0.25
3935679,11764610,61,DE,RT,Goal,Handicap3w,None,FB-219054,2022-10-28,2022-10-28 14:23:39.479,...,DE,Football,2. Bundesliga,126.0,0.50,6,0.25,0.25,0.25,0.25
3935678,11764610,61,DE,RT,Goal,Handicap3w,None,FB-219064,2022-10-28,2022-10-28 14:23:39.479,...,DE,Football,2. Bundesliga,126.0,0.50,6,0.25,0.25,0.25,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,12089410,29,DE,RT,Goal,Handicap2w,None,FB-106204,2022-09-01,2022-09-01 00:00:28.594,...,US,Football,MLS,0.0,8.34,5,3.47,5.83,5.83,8.34
3,8130610,29,DE,RT,Goal,PlayerToScore,None,FB-106204,2022-09-01,2022-09-01 00:00:26.666,...,US,Football,MLS,0.0,15.00,5,8.66,13.00,12.79,15.00
2,12072510,44,DE,RT,Goal,OverUnder,None,FB-106190,2022-09-01,2022-09-01 00:00:22.536,...,UY,Football,Cup,0.0,3.75,5,3.26,3.75,3.75,3.75
1,12072510,44,DE,HT1,Goal,OverUnder,None,FB-106228,2022-09-01,2022-09-01 00:00:22.536,...,US,Football,MLS,0.0,3.75,5,3.29,3.75,3.75,3.75


In [2]:
test_betmarkets = DeltaTable("/Users/andreikirpichev/parquet/test_betmarkets/")
test_betmarkets = test_betmarkets.to_pandas()
test_betmarkets.head(1)

,contest_id,section,point,market,away,start,end,home,league,region,sport,placement_odds
0,FB-216767,RT,goal,BothTeamsToScore,UE Engordany,2022-10-22 14:53:19.559,2022-10-23 16:30:00,UE Santa Coloma,Primera Division,AD,Football,1.72


In [3]:
from sklearn.preprocessing import LabelEncoder
betmarket_categorical_features = ['section', 'point', 'market', 'region', 'contest_id', 'league', 'home', 'away']
categorical_features = betmarket_categorical_features + ['country']

le = LabelEncoder()

# Convert the variables to numerical
def encode(arr, features):
    for i in range(len(features)):
        encoded = le.fit_transform(arr[features[i]])
        arr[features[i]] = encoded

encode(test_betmarkets, betmarket_categorical_features)
encode(bets, categorical_features)

In [4]:
bets['section'].value_counts()

7    3419599
1     480459
2      29604
4       2995
3       1686
8        673
9        438
5        153
6         72
0          4
Name: section, dtype: int64

In [5]:
bets.head(1)

,customer_id,age,country,section,point,market,market_params,contest_id,placement_date,placement_time,...,region,sport,league,minutes_before_start,stake_eur,dayofweek,weekday_stake,section_2w,point_2w,market_2w
0,9786810,26,2,7,1,11,None,161,2022-09-01,2022-09-01 00:00:07.176,...,60,Football,117,60.0,11.25,5,2.98,6.25,6.25,6.56


In [6]:
test_betmarkets.head(1)

,contest_id,section,point,market,away,start,end,home,league,region,sport,placement_odds
0,1502,4,1,0,2504,2022-10-22 14:53:19.559,2022-10-23 16:30:00,2516,127,0,Football,1.72


In [7]:
# Create features and target
features = categorical_features + ['age', 'weekday_stake', 'placement_odds', 'section_2w', 'point_2w', 'market_2w']
print(features)
features_meta = features + ['placement_time', 'customer_id']
X = bets[features_meta]
y = bets[['stake_eur']]

# show rows with NaN values
#print(X[X.isna().any(axis=1)])

# Make necessary imports
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 101, shuffle=False)

print('train: from', X_train['placement_time'].min(), ' to ', X_train['placement_time'].max(), X_train.shape)
print('test: from', X_test['placement_time'].min(), ' to ', X_test['placement_time'].max(), X_test.shape)
threshhold = X_train['placement_time'].max()

X_train = X_train[features]

# Train the model
regr = RandomForestRegressor(n_estimators = 10, max_depth = 10, random_state = 101)
regr.fit(X_train, y_train.values.ravel())

['section', 'point', 'market', 'region', 'contest_id', 'league', 'home', 'away', 'country', 'age', 'weekday_stake', 'placement_odds', 'section_2w', 'point_2w', 'market_2w']
train: from 2022-09-01 00:00:07.176000  to  2022-10-18 02:08:34.266000 (3148546, 17)
test: from 2022-10-18 02:08:40.166000  to  2022-10-28 14:23:42.051000 (787137, 17)


RandomForestRegressor(max_depth=10, n_estimators=10, random_state=101)

In [16]:
X_test.shape

(787137, 17)

In [17]:
X_test.head(5)

,section,point,market,region,contest_id,league,home,away,country,age,weekday_stake,placement_odds,section_2w,point_2w,market_2w,placement_time,customer_id
3148546,7,1,11,87,30911,167,2668,5673,2,54,11.88,1.280,8.45,8.41,9.17,2022-10-18 02:08:40.166,4526010
3148547,7,1,11,87,30919,167,1587,1519,2,54,11.78,1.240,8.45,8.41,9.17,2022-10-18 02:08:40.166,4526010
3148548,7,1,11,122,30742,178,2608,4814,2,54,15.25,1.115,8.45,8.41,9.17,2022-10-18 02:08:40.166,4526010
3148549,7,1,0,120,30962,222,2726,4030,2,30,0.51,1.780,0.57,0.56,0.65,2022-10-18 02:08:41.854,5475010
3148550,7,1,11,41,30134,151,3182,760,2,30,0.39,2.360,0.57,0.56,0.53,2022-10-18 02:08:41.854,5475010


In [18]:
X_test_inv = X_test[:]
encode(X_test_inv, categorical_features)
X_test_inv.head(1)

,section,point,market,region,contest_id,league,home,away,country,age,weekday_stake,placement_odds,section_2w,point_2w,market_2w,placement_time,customer_id
3148546,7,1,11,80,265,98,1286,2752,2,54,11.88,1.28,8.45,8.41,9.17,2022-10-18 02:08:40.166,4526010


In [12]:
def f(r):
    print(X_test_inv[r].shape)
[f(x) for x in X_test_inv[['age', 'weekday_stake', 'section_2w', 'point_2w', 'market_2w']]]

(787137,)
(787137,)
(787137,)
(787137,)
(787137,)


[None, None, None, None, None]

In [ ]:
import warnings
warnings.filterwarnings('ignore')

X_test = X_test.head(1)
# Make prediction
print(X_test[features].shape)
predictions = regr.predict(X_test[features])

result = X_test[:]
result['stake_eur'] = y_test
result['prediction'] = predictions.tolist()
result.head()

In [ ]:
bets[['market', 'point', 'section']].value_counts()

In [25]:
pd.options.mode.chained_assignment = None
size = 1000
shuffled = X_test[:].sample(frac=1)
progress = 0
pred_sum = 0
base_sum = 0
for index, row in shuffled.head(size).iterrows():
    current_betmarkets = test_betmarkets[(test_betmarkets['start'] <= row['placement_time']) &
                                         (test_betmarkets['end'] >= row['placement_time'])]
    customer_bet_features = ['country', 'age', 'weekday_stake']#, 'section_2w', 'point_2w', 'market_2w']
    current_betmarkets[customer_bet_features] = row[customer_bet_features]
    bets_2w = bets[(bets['placement_time'] > row['placement_time'] - np.timedelta64(14,'D')) &
                   (bets['placement_time'] < row['placement_time'])]
    def set_avg_2w(bm_type):
        type_2w = bets_2w.groupby(['customer_id',bm_type],as_index=False)['stake_eur'].mean('')
        type_by_customer = type_2w[type_2w['customer_id'] == row['customer_id']][[bm_type, 'stake_eur']]
        current_betmarkets[bm_type + '_2w'] = pd.merge(current_betmarkets, type_by_customer, on=bm_type)[['stake_eur']]
        current_betmarkets[bm_type + '_2w'] = current_betmarkets[bm_type + '_2w'].fillna(0)

    set_avg_2w('section')
    set_avg_2w('point')
    set_avg_2w('market')

    predictions = regr.predict(current_betmarkets[features])
    current_betmarkets['prediction'] = predictions.tolist()
    recommended = current_betmarkets.query('prediction == prediction.max()')
    model_predicted = recommended.iloc[0]

    customer_bets = bets_2w[bets_2w['customer_id'] == row['customer_id']]
    if customer_bets.empty:
        baseline_predicted['market'] = 4
        baseline_predicted['point'] = 7
        baseline_predicted['section'] = 37
    else:
        last_customer_bet = customer_bets.query('placement_time == placement_time.max()')
        baseline_predicted = last_customer_bet.iloc[0]

    customerContestIdInteractions = bets_2w[(bets_2w['customer_id'] == row['customer_id']) &
                                            (bets_2w['contest_id'] == row['contest_id'])]
    if customerContestIdInteractions.empty:
        mostPopular = bets_2w[['contest_id', 'region', 'league']].mode().iloc[0]
        baseline_predicted[['region', 'league']] = mostPopular[['region', 'league']]
    else:
        baseline_predicted[['region', 'league']] = customerContestIdInteractions.iloc[0]


    def hitRate(predicted, sum):
        res = ((1 if predicted['league'] == row['league'] else 0) +
               (1 if predicted['region'] == row['region'] else 0) +
               (1 if predicted['market'] == row['market'] else 0) +
               (1 if predicted['point'] == row['point'] else 0) +
               (1 if predicted['section'] == row['section'] else 0)) / 5
        sum += res
        return sum

    progress += 1
    if progress % 50 == 0:
        print("Mode Hit Rate: ", round(hitRate(model_predicted, pred_sum) / progress * 100, 2), "%",
              "baseline: ", round(hitRate(baseline_predicted, base_sum) / progress * 100, 2), "%",
              " processed ", round(progress / size * 100, 2), "%")

Mode Hit Rate:  0.4 % baseline:  0.8 %  processed  5.0 %
Mode Hit Rate:  0.0 % baseline:  0.8 %  processed  10.0 %
Mode Hit Rate:  0.13 % baseline:  0.27 %  processed  15.0 %
Mode Hit Rate:  0.1 % baseline:  0.5 %  processed  20.0 %
Mode Hit Rate:  0.08 % baseline:  0.4 %  processed  25.0 %
Mode Hit Rate:  0.07 % baseline:  0.33 %  processed  30.0 %
Mode Hit Rate:  0.06 % baseline:  0.17 %  processed  35.0 %
Mode Hit Rate:  0.1 % baseline:  0.15 %  processed  40.0 %
Mode Hit Rate:  0.04 % baseline:  0.13 %  processed  45.0 %
Mode Hit Rate:  0.04 % baseline:  0.2 %  processed  50.0 %
Mode Hit Rate:  0.04 % baseline:  0.07 %  processed  55.0 %
Mode Hit Rate:  0.03 % baseline:  0.07 %  processed  60.0 %
Mode Hit Rate:  0.06 % baseline:  0.09 %  processed  65.0 %
Mode Hit Rate:  0.03 % baseline:  0.11 %  processed  70.0 %
Mode Hit Rate:  0.03 % baseline:  0.11 %  processed  75.0 %
Mode Hit Rate:  0.03 % baseline:  0.03 %  processed  80.0 %
Mode Hit Rate:  0.05 % baseline:  0.09 %  processe

In [ ]:
# Import library for metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Mean absolute error (MAE)
mae = mean_absolute_error(y_test.values.ravel(), predictions)

# Mean squared error (MSE)
mse = mean_squared_error(y_test.values.ravel(), predictions)

# R-squared scores
r2 = r2_score(y_test.values.ravel(), predictions)

# Print metrics
print('Mean Absolute Error:', round(mae, 2))
print('Mean Squared Error:', round(mse, 2))
print('R-squared scores:', round(r2, 2))

In [ ]:
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV

# Find the best parameters for the model
parameters = {
    'max_depth': [70, 80, 90, 100],
    'n_estimators': [900, 1000, 1100]
}
gridforest = GridSearchCV(regr, parameters, cv = 3, n_jobs = -1, verbose = 1)
gridforest.fit(X_train, y_train)
gridforest.best_params_

In [ ]:
# Get features list
characteristics = X.columns
# Get the variables importances, sort them, and print the result
importances = list(regr.feature_importances_)
characteristics_importances = [(characteristic, round(importance, 2)) for characteristic, importance in zip(characteristics, importances)]
characteristics_importances = sorted(characteristics_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in characteristics_importances];

In [ ]:
# Import library for visualization
import matplotlib.pyplot as plt
# Visualize the variables importances
plt.bar(characteristics, importances, orientation = 'vertical')
plt.xticks(rotation = 'vertical')
plt.ylabel('Importance')
plt.xlabel('Variable')
plt.grid(axis = 'y', color = '#D3D3D3', linestyle = 'solid')
plt.show()

In [ ]:
pd.options.mode.chained_assignment = None
size = 2000
sum = 0.0
shuffled = X_test[:].sample(frac=1)
progress = 0
for index, row in shuffled.head(size).iterrows():
    current_betmarkets = test_betmarkets[(test_betmarkets['start'] <= row['placement_time']) &
                                         (test_betmarkets['end'] >= row['placement_time'])]
    customer_bet_features = ['country', 'age', 'weekday_stake']#, 'section_2w', 'point_2w', 'market_2w']
    current_betmarkets[customer_bet_features] = row[customer_bet_features]
    bets_2w = bets[(bets['placement_time'] >= row['placement_time'] - np.timedelta64(14,'D')) &
                        (bets['placement_time'] <= row['placement_time'])]
    def set_avg_2w(bm_type):
        type_2w = bets_2w.groupby(['customer_id',bm_type],as_index=False)['stake_eur'].mean('')
        type_by_customer = type_2w[type_2w['customer_id'] == row['customer_id']][[bm_type, 'stake_eur']]
        current_betmarkets[bm_type + '_2w'] = pd.merge(current_betmarkets, type_by_customer, on=bm_type)[['stake_eur']]
        current_betmarkets[bm_type + '_2w'] = current_betmarkets[bm_type + '_2w'].fillna(0)

    set_avg_2w('section')
    set_avg_2w('point')
    set_avg_2w('market')

    predictions = regr.predict(current_betmarkets[features])
    current_betmarkets['prediction'] = predictions.tolist()
    #print(len(current_betmarkets))
    recommended = current_betmarkets.query('prediction == prediction.max()')
    predicted = recommended.iloc[0]
    res = ((1 if predicted['league'] == row['league'] else 0) +
           (1 if predicted['region'] == row['region'] else 0) +
           (1 if predicted['market'] == row['market'] else 0) +
           (1 if predicted['point'] == row['point'] else 0) +
           (1 if predicted['section'] == row['section'] else 0)) / 5
    progress += 1
    sum += res
    if progress % 100 == 0:
        print(progress / size * 100, "% processed with Hit Rate: ", sum / progress * 100, "%")

print("Result:", sum / size)


#print(row.shape, row)